In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

import requests
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

In [2]:
model_name = "meta-llama/Llama-3.2-11B-Vision-Instruct"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name,
model = MllamaForConditionalGeneration.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )
processor = AutoProcessor.from_pretrained(model_name)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [105]:
import pandas as pd
import os

# 指定user文件夹路径
user_folder = "user"  

# 创建一个字典存储所有df
dfs = {}

# 遍历user文件夹下的文件
for i, filename in enumerate(os.listdir(user_folder)):
    if filename.endswith(".csv"):
        # 读取每个csv文件
        filepath = os.path.join(user_folder, filename)
        dfs[i+1] = pd.read_csv(filepath)
        print(i+1)
        print(filename)
        #print(dfs[i+1].head())  # 打印前5行以确认读取成功
else:
    if not dfs:  # 判断dfs字典是否为空
        print("未在user文件夹中找到csv文件。")

1
1040548857_宛沂.csv
2
1218786407_记住开心.csv
3
1017943193_写代码的Lucian.csv
4
1215051697_時光記憶SZ.csv
5
1168582244_指点郑州.csv
7
1735661432_翔较瘦.csv
8
1735479231_飞扬1735479231.csv
9
1439228800_云墨咕策.csv
10
1747731124_坚强的haozi.csv
11
1590831675_深圳赵国忠.csv
12
1553170053_我是不是安静的男子.csv
13
1743567455_緈諨儛埗.csv
14
1400515853_红黑骑士ing.csv


In [84]:
import requests

def download_image(image_url, save_path):
    # 发送HTTP GET请求获取图片内容
    response = requests.get(image_url)

    # 检查请求是否成功
    if response.status_code == 200:
        # 以二进制写入方式打开本地文件
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image saved to {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

# 示例用法
#image_url = 'https://ww1.sinaimg.cn/large/486c37b1gw6dbeejrajl1j.jpg'
save_path = 'downloaded_image.jpg'

Image saved to downloaded_image.jpg


In [132]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()

In [96]:
with open('prompt_poi.txt', 'r') as file:
    prompt = file.read()

In [122]:
with open('prompt_img.txt', 'r') as file:
    prompt = file.read()

In [123]:
output_string = ""

for index, row in dfs[4].iloc[2:3].iterrows():
    output_string += f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}\n"
    img_url = row['img_urls']
    #output_str = f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}"
    #print(output_str)
#print(output_string)

download_image(img_url, save_path)
new_text = prompt + '\n\nContents: \n' + output_string
print(new_text)

Image saved to downloaded_image.jpg
The following is a record of a user posting on social media, which contains coordinate information, but the coordinates are likely to be wrong. Analyze the user's real location and what he was doing at the time based on the text content and pictures. The location should be as detailed as possible, and the coordinates must be given, including the province and city as much as possible.

Contents: 
2010-11-15 10:22:00, 无 (0,0), 刚从西藏拍的，发一张试试。  原图 



In [125]:
image = Image.open(save_path)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=256)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>The following is a record of a user posting on social media, which contains coordinate information, but the coordinates are likely to be wrong. Analyze the user's real location and what he was doing at the time based on the text content and pictures. The location should be as detailed as possible, and the coordinates must be given, including the province and city as much as possible.

Contents: 
2010-11-15 10:22:00, 无 (0,0), 刚从西藏拍的，发一张试试。  原图 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The user is likely in the city of Lhasa, Tibet, China. The image shows a monk standing on a bridge, with a large number of buildings in the background, which is consistent with the architecture of Lhasa. The coordinates provided, (0,0), are likely incorrect and should be ignored.

The user was likely taking a photo of the monk and the surrounding area, possibly as part of a personal project or to share with others on social

In [134]:
output_string = ""

for index, row in dfs[4].iloc[0:10].iterrows():
    output_string += f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}\n"
    #output_str = f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}"
    #print(output_str)
#print(output_string)

new_text = prompt + '\n\nContents: \n' + output_string
print(new_text)

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographical proximity to infer the user's full route and potential stops even if they didn't explicitly post from those locations.

User Profile: 
1. 居住地：
根据用户的帖子内容和标签，我推测用户可能居住在深圳市。他们在

In [135]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=1024)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [131]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=1024)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [111]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [78]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

The following is a record of posts published by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post. Your task is to find the user's points of interest, including but not limited to the location of the user's home, workplace, school, places they often go to, and the coordinate types of these places. The location should be as detailed as possible, the coordinates must be given, and the province and city should be included as much as possible.

Example Output:
The user likely lives in 广州 (23.10809, 113.26467).
The user likely to be a student at 武汉科技大学 (114.26727, 30.44067).
The user traveled to 九寨沟 (33.33333, 103.83333) to take photos, as mentioned in the post from 2010-11-15.

Contents: 
2011-01-06 14:47:00, 无 (0,0), 九寨回来了。运气依然谈很好。想雪就下了点雪，想晴，就出来

In [51]:
output = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - (114.397436, 30.477216)Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographical p